### Multiple Mutect에서 shared call 구하기

In [4]:
import vcf
import sys
import os.path
import argparse

DIR = "/home/goldpm1/Meningioma/04.mutect/03.vep"

MIN_BLOCK_SHARED = 1
print ("MIN_BLOCK_SHARED = {}".format(MIN_BLOCK_SHARED))

for INPUT_VCF in [ DIR + "/220930_multiple.MT2.FMC.HF.RMBLACK.vep.vcf", DIR + "/221026_multiple.MT2.FMC.HF.RMBLACK.vep.vcf", DIR + "/221102_multiple.MT2.FMC.HF.RMBLACK.vep.vcf"]:
#for INPUT_VCF in [ DIR + "/220930_Dura.MT2.FMC.HF.RMBLACK.vep.vcf", DIR + "/221026_Dura.MT2.FMC.HF.RMBLACK.vep.vcf", DIR + "/221102_Dura.MT2.FMC.HF.RMBLACK.vep.vcf",  DIR + "/220930_Tumor.MT2.FMC.HF.RMBLACK.vep.vcf", DIR + "/221026_Tumor.MT2.FMC.HF.RMBLACK.vep.vcf", DIR + "/221102_Tumor.MT2.FMC.HF.RMBLACK.vep.vcf"]:
    print ("\n\n" + INPUT_VCF)
    vcf_reader = vcf.Reader(open(INPUT_VCF, "r"))
    samplenames = vcf_reader.samples        # ['220930_Blood', '220930_Dura', '220930_Tumor']

    Venn_data = {}
    first_line = ["CHR:POS", "Gene", "Ref", "Alt", "Variant_Classification"]
    for sampleindex, samplename in enumerate(vcf_reader.samples):
        Venn_data[samplename] = []
        if "Blood" not in samplename:
            first_line.append (samplename)          # ['CHR', 'POS', '220930_Dura', '220930_Tumor']


    print (*first_line, sep = "\t")

    variant_count_total = 0

    for record in vcf_reader:        # record.CHROM, recrod.POS ,record.ALT
        if record.CHROM == "chrM":
            break
        # variant가 몇 sample에서 나오는지 찾기
        count = 0
        for sampleindex, samplename in enumerate(samplenames):    # ['220930_Dura', '220930_Tumor']
            
            if "Blood" not in samplename:
                if record.samples[sampleindex]["GT"] not in ["./.", "0/0", "0|0"] :   # Genotype 조건 만족
                    if None not in record.samples[sampleindex]["AD"]:      # 왜인지는 모르겠으나 [25, 5, None] 같은 경우도 있다
                        if record.samples[sampleindex]["AD"][-1] != 0:
                            #print (record.CHROM, record.POS, samplename, record.samples[sampleindex]["GT"], record.samples[sampleindex]["AD"])
                            count = count  + 1


        if count >= MIN_BLOCK_SHARED:
            try:
                Gene_Symbol = (record.INFO["CSQ"][0].split("|")[3])       # 일단 맨 상위 script만 보자
                if record.INFO["CSQ"][0].split("|")[1] == "intergenic_variant":
                    Gene_Symbol = "intergenic_variant"
                elif record.INFO["CSQ"][0].split("|")[1] == "regulatory_region_variant":
                    Gene_Symbol = "regulatory_region_variant"
            except:
                Gene_Symbol = ""
                
            line = [str(record.CHROM) + ":" + str(record.POS), str(Gene_Symbol), str(record.REF), str(record.ALT)[1:-1], record.INFO["CSQ"][0].split("|")[1] ]
            #line = [str(record.CHROM) + ":" + str(record.POS), str(Gene_Symbol) ]
            
            #print ( record.INFO["CSQ"][0].split("|") ) 
            
            check = 0
            for sampleindex, samplename in enumerate(samplenames):
                if "Blood" not in samplename:
                    line.append('.'.join([str(a) for a in record.samples[sampleindex]["AD"]]))       # "REF.ALT"
                    if (0 in record.samples[sampleindex]["AD"]) | (  int ( record.samples[sampleindex]["AD"][1] )  < 2):
                        check = 1
        
            
            if check == 0:
                print ("\t".join(line))
                variant_count_total = variant_count_total + 1


    print ("총 Variant = {}개".format(variant_count_total))
    # output_file.close()
    # os.system("cp {0} {1}".format(OUTPUT + "_" + str(MIN_BLOCK_SHARED) + ".txt", OUTPUT2 + "_" + str(MIN_BLOCK_SHARED) + ".txt"))

    #Call(sample=220930_Tumor, CallData(GT=0/1, AD=[6, 0], AF=[0.143], DP=6, F1R2=[2, 0], F2R1=[3, 0], FAD=[5, 0], SB=[3, 3, 0, 0]))

MIN_BLOCK_SHARED = 1


/home/goldpm1/Meningioma/04.mutect/03.vep/220930_multiple.MT2.FMC.HF.RMBLACK.vep.vcf
CHR:POS	Gene	Ref	Alt	Variant_Classification	220930_Dura	220930_Tumor
chr1:44802872	TCTEX1D4	A	C	downstream_gene_variant	98.14	26.13
chr1:150487535	TARS2	A	G	intron_variant&NMD_transcript_variant	124.8	64.26
chr1:243221776	CEP170	T	C	missense_variant	492.48	224.115
chr2:3644016	COLEC11	C	T	synonymous_variant	830.63	440.139
chr2:44706287	CAMKMT	G	C	missense_variant&splice_region_variant	278.16	140.39
chr2:178554137	TTN	A	T	synonymous_variant	603.19	323.84
chr3:11817144	TAMM41	T	A	intron_variant	363.20	238.91
chr3:49721007	GMPPB	C	T	3_prime_UTR_variant	397.25	241.83
chr3:50194058	SEMA3F	C	A	downstream_gene_variant	150.8	72.36
chr3:52322739	DNAH1	C	T	synonymous_variant	476.2	338.21
chr5:71656712	MCCC2	A	T	intron_variant	174.8	88.28
chr5:103108723	GIN1	A	G	intron_variant	211.11	127.32
chr6:32554712	HLA-DRB6	C	T	non_coding_transcript_exon_variant	76.8	51.2
chr6:87255842	ZNF292	A	G	miss

In [ ]:
df = pd.read_csv("/data/project/Alzheimer/Public/2022GBM/WGS/04.Analysis/sequenza_shared/WGS_cnv_input.txt", sep = "\t")
df.head()